In [5]:
import torch
import numpy
from torch import nn
import torchvision
from torchvision import transforms, datasets
from torch import optim

In [6]:
import torch.nn.functional as F

In [9]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,),(0.5,)),
                              ])

In [13]:
trainset= datasets.MNIST('~/.pytorch/MNIST_data//',download=True,train=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)


# for images,label in trainloader:
#     print(images.shape)
#     break
    
# Output:
# torch.Size([64, 1, 28, 28])

In [11]:
testset= datasets.MNIST('~/.pytorch/MNIST_data//',download=True,train=True,transform=transform)
testloader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)

In [14]:
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(784,128)
        self.fc2 = torch.nn.Linear(128,64)
        self.fc3 = torch.nn.Linear(64,10)
        self.sigmoid = torch.nn.Sigmoid()
        self.softmax = torch.nn.Softmax(dim=1)
        self.relu = torch.nn.ReLU()
    def forward(self,x):
        # x is of size (64,1,28,28)
        # reshape to (64,784)
        x = x.view(x.shape[0],-1)
        x = self.fc1.forward(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x
        

In [15]:
model = MLP()

criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr = 0.05)

In [22]:
epochs = 5
for e in range(epochs):
    print('Epoch {}'.format(e+1))
    running_loss = 0
    for images,labels in trainloader:
        optimizer.zero_grad()
        logits = model.forward(images)
        loss = criterion(logits,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        accuracy = 0
        with torch.no_grad():
            for images,labels in testloader:
                logit = model(images)
                ps = logit #torch.exp(logit)
                top_p,top_class = ps.topk(1,dim = 1)
                equals = top_class == labels.view(*top_class.shape)
#                 accuracy += torch.mean(equals.type(torch.FloatTensor))
                accuracy += torch.mean(equals.type(torch.FloatTensor))                
        print('Train accuracy: {}'.format(running_loss))
        print('Test accuracy: {}'.format((accuracy/len(testloader)).data.numpy()))bb

Epoch 1
Train accuracy: 1423.7482254505157
Test accuracy: 0.9515591859817505
Epoch 2
Train accuracy: 1420.9506032466888
Test accuracy: 0.9500433206558228
Epoch 3
Train accuracy: 1418.1968749761581
Test accuracy: 0.9564065933227539
Epoch 4
Train accuracy: 1415.913057088852
Test accuracy: 0.9565231800079346
Epoch 5
Train accuracy: 1413.7769304513931
Test accuracy: 0.9570229053497314
